In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
import os
import pandas as pd
import numpy as np
import cv2

In [2]:
base_dir = r'E:\AB\UEM\year 3\Project\Sem5\cov_deep_learning\Data_dl'
img_h,img_w= (164,164)
batch_size=256
epochs=3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen= ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [3]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.callbacks import ReduceLROnPlateau

base_model= ResNet50(include_top=False, weights='imagenet',
                                       input_tensor=None, input_shape=(img_h,img_w,3), pooling='avg')
 
for layer in base_model.layers[:-7]:
    layer.trainable=False

 
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.4))


model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 1,059,842
Non-trainable params: 22,531,968
_________________________________________________________________


In [4]:
for layer in base_model.layers:
    print(layer,"-->",layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x0000021CB2579C70> --> False
<keras.layers.convolutional.ZeroPadding2D object at 0x0000021CB25D2430> --> False
<keras.layers.convolutional.Conv2D object at 0x0000021CB25D27C0> --> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000021CB26475E0> --> False
<keras.layers.core.activation.Activation object at 0x0000021CB26472B0> --> False
<keras.layers.convolutional.ZeroPadding2D object at 0x0000021CB2682BE0> --> False
<keras.layers.pooling.MaxPooling2D object at 0x0000021CB26D5B20> --> False
<keras.layers.convolutional.Conv2D object at 0x0000021CB26F5C40> --> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000021CB26F2FA0> --> False
<keras.layers.core.activation.Activation object at 0x0000021CB26F5430> --> False
<keras.layers.convolutional.Conv2D object at 0x0000021CB26FFA60> --> False
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000

In [5]:
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau


reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=3,
                                         cooldown=2,
                                         min_lr=1e-10,
                                         verbose=1)

checkpoint =tf.keras.callbacks.ModelCheckpoint(filepath="resfinal.h5", 
                            monitor='val_accuracy',
                            verbose=1,
                            save_best_only=True, 
                            save_weights_only=False, 
                            mode='auto',
                            save_freq='epoch')


import time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

callbacks = [reduce_learning_rate, checkpoint, time_callback]
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile( loss='categorical_crossentropy',optimizer= optimizer, metrics=['accuracy'])

F:\Anaconda\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [6]:
train_generator = datagen.flow_from_directory(
        base_dir,  # This is the source directory for training images
        target_size=(img_h,img_w),  
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        #color_mode="rgb",
        shuffle=True)


validation_generator = datagen.flow_from_directory(
         base_dir,     
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='categorical',
        #color_mode="rgb",
        subset='validation',
        shuffle=False)

Found 1986 images belonging to 2 classes.
Found 495 images belonging to 2 classes.


In [20]:
model.compile(optimizer='sgd',
          loss='binary_crossentropy',
          metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall()])
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//batch_size, 
      epochs=1,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//batch_size,  
      callbacks=callbacks,
      verbose=1)

7/7 [==============================] - ETA: 0s - loss: 0.6154 - accuracy: 0.6682 - precision_1: 0.6682 - recall_1: 0.6682 WARNING:tensorflow:6 out of the last 8 calls to <function Model.make_test_function.<locals>.test_function at 0x0000021CC0A07550> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.

Epoch 00001: val_accuracy did not improve from 0.97656
7/7 [==============================] - 139s 20s/step - loss: 0.6154 - accuracy: 0.6682 -

In [27]:
model.evaluate(validation_generator,verbose=1)

2/2 [==============================] - 33s 14s/step - loss: 0.9147 - accuracy: 0.5313


[0.914727509021759, 0.5313131213188171]

In [19]:
history_dict = history.history
print(history_dict.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'lr'])


In [82]:
img = cv2.imread("E:\\AB\\UEM\\year 3\\Project\\Sem5\\cov_deep_learning\\Data_dl\\non-COVID\\Non-Covid (28).png") 
image_resized= cv2.resize(img, (img_h,img_w))
cv2.imshow('CT scan', img)
cv2.waitKey(0)
img=np.expand_dims(image_resized,axis=0)
np.set_printoptions(suppress=True)
pred=model.predict(img)
print(pred)

[[0.16584957 0.83415043]]


In [72]:
yn=np.argmax(pred,axis=1)
classes = []
for subdir in sorted(os.listdir(base_dir)):
    if os.path.isdir(os.path.join(base_dir, subdir)):
        classes.append(subdir)
if yn==0:
    print("COVID positive")
else :
    print("Not Covid")

COVID positive


In [15]:
yn=np.argmax(pred,axis=1)
classes = []
for subdir in sorted(os.listdir(base_dir)):
    if os.path.isdir(os.path.join(base_dir, subdir)):
        classes.append(subdir)
yn

array([0], dtype=int64)

In [48]:
import keras
from keras.utils.np_utils import to_categorical
y_proba = model.predict(img)
y_classes = y_proba.argmax(axis=-1)
y_classes

array([0], dtype=int64)

In [47]:
timings= time_callback.times
n=sum(timings)
print("Total time taken is ",sum(timings)//60," minutes and ",sum(timings)%60," seconds")

Total time taken is  6.0  minutes and  15.173778057098389  seconds


In [48]:
model2= tf.keras.models.load_model("resfinal.h5")
y_pred= model2.predict(validation_generator,verbose=1)
y_true= validation_generator.classes
y_pred2= np.argmax(y_pred,axis=1)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_true,y_pred2))
from sklearn.metrics import confusion_matrix

cf=confusion_matrix(y_true, y_pred2)

df_cm = pd.DataFrame(cf, range(2), range(2))
df_cm.to_csv('ResNet50 prj'+'.csv',index=False)

2/2 [==============================] - 22s 12s/step
0.5313131313131313
